<div style='background-color: #ffc154;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Devoir maison</h2>
    <h1>Séismes</h1>
</div>

L'Institut d'études géologiques des États-Unis (_United States Geological Survey_) met à disposition une API permettant de récupérer des données sur les séismes. En envoyant une requête à l'URL [https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=4](https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=4), l'API renvoie les données correspondant aux séismes survenus pendant les 30 derniers jours et dont la magnitude est supérieure à 4 sur l'échelle de Richter.

**(1)** Écrire des lignes de code permettant d'interroger l'API et d'afficher à l'écran le nombre de séismes, survenus pendant les 30 derniers jours, dont la magnitude est supérieure à 6.

In [ ]:
import requests

In [ ]:
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=6"
reponse = requests.get(url)
reponse = reponse.json()
print(reponse["metadata"]["count"])

La documentation de l'API est consultable à l'adresse [https://earthquake.usgs.gov/fdsnws/event/1/](https://earthquake.usgs.gov/fdsnws/event/1/).

**(2)** Après avoir consulté la documentation de l'API, recopier et modifier les lignes de code précédentes dans le but d'interroger l'API et d'afficher à l'écran le nombre de séismes :
- survenus entre le 1er janvier 2024 et le 30 juin 2024, et
- survenus entre les latitudes 34°N et 72°N et les longitudes 25°O et 55°E, ce qui correspond à peu près à l'Europe, et
- dont la magnitude est supérieure à 4.

In [ ]:
import requests
url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude=4"
url += "&starttime=2024-01-01&endtime=2024-06-30"
url += "&minlatitude=34&maxlatitude=72&minlongitude=-25&maxlongitude=55"
reponse = requests.get(url)
reponse = reponse.json()
print(reponse["metadata"]["count"])

**(3)** Définir une fonction `interroger_API_seismes` qui prend en entrée deux chaînes de caractères `debut` et `fin` au format `'AAAA-MM-JJ'` et un nombre `mag_min` et interroge l'API à propos des séismes survenus entre la date `debut` et la date `fin` et dont la magnitude est supérieure à `mag_min`.

In [ ]:
def interroger_API_seismes(debut, fin, mag_min):
    """
    Interroge l'API de l'Institut d'études géologiques des États-Unis à propos des séismes survenus
    sur une période de temps et dont la magnitude est supérieure à une valeur donnée.
    - Entrées : debut, fin (chaînes de caractères au format 'AAAA-MM-JJ', début et fin de la période de temps),
                mag_min (nombre, magnitude minimale)
    - Sortie : reponse (dictionnaire)
    """
    url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&minmagnitude={mag_min}"
    url += f"&starttime={debut}&endtime={fin}"
    reponse = requests.get(url)
    reponse = reponse.json()
    return reponse

**(4)** Compléter la définition de la fonction `carte_seismes`.

In [ ]:
import folium

In [ ]:
def carte_seismes(debut, fin, mag_min):
    """
    Place sur une carte la localisation des séismes survenus sur une période de temps
    et dont la magnitude est supérieure à une valeur donnée.
    - Entrées : debut, fin (chaînes de caractères au format 'AAAA-MM-JJ', début et fin de la période de temps),
                mag_min (nombre, magnitude minimale)
    - Sortie : carte (instance de la classe Map du module folium)    
    """
    carte = folium.Map(location=(0, 0), tiles="OpenStreetMap", zoom_start=2)
    donnees = interroger_API_seismes(debut, fin, mag_min)
    for elem in donnees["features"]:
        mag = elem["properties"]["mag"]
        lat = elem["geometry"]["coordinates"][1]
        long = elem["geometry"]["coordinates"][0]
        folium.CircleMarker(location=(lat, long), radius=3, color="black").add_to(carte)
    return carte

**(5)** Afficher la carte des séismes de magnitude au moins 4 survenus au cours des mois de juillet et août 2024.

In [ ]:
carte_seismes("2024-07-01", "2024-08-31", 4)

**(6)** Recopier et modifier la définition de la fonction `carte_seismes` dans le but de représenter chaque séisme par un cercle dont la couleur dépend de la magnitude :

| Magnitude       | Couleur      |
| :-------------: |:------------:|
| Supérieure à 8  | _rouge_      |
| Entre 7 et 8    | _orange_     |
| Entre 6 et 7    | _jaune_      |
| Entre 5 et 6    | _vert_       |
| Entre 4 et 5    | _vert clair_ |

_Remarque :_ Représenter les séismes par ordre croissant de magnitude permet de mettre en évidence ceux dont la magnitude est la plus grande.

In [ ]:
COULEURS = {4: "lightgreen", 5: "green", 6: "yellow", 7: "orange", 8: "red", 9: "red", 10: "red"}

def cle_magnitude(d):              # Cette fonction sera utilisée comme clé de tri
    return d["properties"]["mag"]  # lors d'un appel à la fonction sorted

def carte_seismes(debut, fin, mag_min):
    """
    Place sur une carte la localisation des séismes survenus sur une période de temps
    et dont la magnitude est supérieure à une valeur donnée.
    - Entrées : debut, fin (chaînes de caractères au format 'AAAA-MM-JJ', début et fin de la période de temps),
                mag_min (nombre, magnitude minimale)
    - Sortie : carte (instance de la classe Map du module folium)    
    """
    carte = folium.Map(location=(0, 0), tiles="OpenStreetMap", zoom_start=2)
    donnees = interroger_API_seismes(debut, fin, mag_min)
    for elem in sorted(donnees["features"], key=cle_magnitude):  # données triées par magnitude croissante
        mag = elem["properties"]["mag"]
        lat = elem["geometry"]["coordinates"][1]
        long = elem["geometry"]["coordinates"][0]
        folium.CircleMarker(location=(lat, long), radius=3, color=COULEURS[int(mag)]).add_to(carte)
    return carte

**(7)** Tester la fonction précédente en affichant (en couleurs) :
- la carte des séismes de magnitude au moins 4 survenus au cours des mois de juillet et août 2024.

In [ ]:
carte_seismes("2024-07-01", "2024-08-31", 4)

- la carte des séismes de magnitude au moins 6 survenus au cours des cinquante dernières années.

In [ ]:
carte_seismes("1974-01-01", "2023-12-31", 6)